In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import config

In [2]:
url = config.INPUT_DATA_URL

In [3]:
# simple try to get text information from the website
# target values: quote, author's name, tags

#collecting raw data in an empty dictionary with keys = id number, and three values = quote, name, tags
final_dict = {}
id = 0

# variable for the next loop - number of pages on the checking website
max_pages = 10

#loop for checking every page of the website
for page in range(max_pages):
    current_url = url + str(page+1)

    #get a page and scrap it with BeautifulSoup
    html_text = requests.get(current_url).text
    soup = BeautifulSoup(html_text, 'lxml')
    base = soup.find_all('div', class_ = 'quote')

    #get quote, author's name and tags from raw data and save it to the empty dictionary
    for i in range(len(base)):
        new_info = base[i]
        id += 1
        final_dict[id] = {}

        quote = new_info.find('span', class_ = "text").text
        author = new_info.find('small').text
        tags = new_info.find_all('a', class_ = 'tag')
        tag_list = []
        for j in range(len(tags)):
            tag_list.append(tags[j].text)
        tag_str = ", ".join(tag_list)

        final_dict[id]["quote"] = quote
        final_dict[id]["author"] = author
        final_dict[id]["tags"] = tag_str.upper()

In [4]:
result = []
current_row = 1

# converting dictionary data to list format for easier output
for i in range(1, len(final_dict) + 1):
    result.append([])

    result[i-1].append(str(final_dict[i]['quote']))
    result[i-1].append(str(final_dict[i]['author']))
    result[i-1].append(str(final_dict[i]['tags']))

In [5]:
# replacing ";" with "," to avoid mistakes with writing to csv file
for i in range(len(result)):
    if ";" in result[i][0]:
        result[i][0] = result[i][0].replace(';', ',')

In [6]:
#framing data with pandas module
df = pd.DataFrame(result, columns=["Quote", "Author", "Tags"])

In [7]:
#saving data to csv file
filename = config.OUTPUT_PATH
df.to_csv(filename)